In [1]:
import logging
import pickle
import numpy as np
from openmmtools.integrators import PeriodicNonequilibriumIntegrator
from simtk import unit, openmm
import argparse
import os
import time
import mdtraj as md
from tqdm import tqdm_notebook
from openmmtools.constants import kB

# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)



In [2]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/15/92/"
phase = 'apo'
sim_number = 0
old_aa_name = 'ala'
new_aa_name = 'tyr'
length = 1
cache = 1
cache_length = 1

## Run neq integrator

In [3]:
# Define lambda functions
x = 'lambda'
inflection1, inflection2, inflection3, inflection4 = 0.2, 0.4, 0.6, 0.8
DEFAULT_ALCHEMICAL_FUNCTIONS = {
                             'lambda_sterics_core': x,
                             'lambda_electrostatics_core': x,
                             'lambda_sterics_insert': f"select(step({x} - {inflection3}), select(step({x}-{inflection4}), 1, (1/{inflection1})*({x}-{inflection3})), 0.0)",
                             'lambda_sterics_delete': f"select(step({x} - {inflection1}), select(step({x} - {inflection2}), 1, (1/{inflection1})*({x}-{inflection1})), 0.0)",
                             'lambda_electrostatics_insert': f"select(step({x} - {inflection4}), (1/{inflection1}) * ({x} - {inflection4}), 0.0)",
                             'lambda_electrostatics_delete': f"select(step({x} - {inflection1}), 1.0, (1/{inflection1})*{x})",
                             'lambda_bonds': x,
                             'lambda_angles': x,
                             'lambda_torsions': x}
# Define simulation parameters
nsteps_eq = 250000  
nsteps_neq = 25000 # 100 ps
neq_splitting='V R H O R V'
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'
cache_length = 1
temperature = 298.0 * unit.kelvin
kT = kB * temperature


In [4]:
# Read in vanilla htf
i = os.path.basename(os.path.dirname(outdir))
with open(os.path.join(outdir, f"{i}_{phase}.pickle"), 'rb') as f:
    htf = pickle.load(f)




In [5]:
# Read in lambda = 0 cache
with open(os.path.join(outdir, f"{i}_{phase}_{old_aa_name}_{cache_length}ns_snapshots.npy"), 'rb') as f:
    subset_pos = np.load(f)
positions = subset_pos[sim_number]
system = htf.hybrid_system

# Read in lambda = 0 cache box vectors
with open(os.path.join(outdir, f"{i}_{phase}_{old_aa_name}_{cache_length}ns_box_vectors.npy"), 'rb') as f:
    subset_box_vectors = np.load(f)
box_vectors = subset_box_vectors[sim_number][0]


In [6]:
# Set up integrator
integrator = PeriodicNonequilibriumIntegrator(DEFAULT_ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep, temperature=temperature)

# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*box_vectors)
context.setPositions(positions)
context.setVelocitiesToTemperature(temperature)



In [7]:
# Run eq forward (0 -> 1)
for _ in tqdm_notebook(range(int(nsteps_eq / 250))):
    integrator.step(250)
pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)



<ipython-input-7-e3738206728a>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm_notebook(range(int(nsteps_eq / 250))):


  0%|          | 0/1000 [00:00<?, ?it/s]

In [8]:
array_sum = np.sum(pos)
array_has_nan = np.isnan(array_sum)
array_has_nan

False

In [9]:
# Run neq forward (0 -> 1)
forward_works_master = list()
forward_neq_old, forward_neq_new = list(), list()
forward_works = [integrator.get_protocol_work(dimensionless=True)]
all_pos = []
for _ in tqdm_notebook(range(int(nsteps_neq / 250))):
    integrator.step(250)
    pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
    all_pos.append(pos)

<ipython-input-9-41b56668e3ed>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm_notebook(range(int(nsteps_neq / 250))):


  0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
array_sum = np.sum(all_pos)
array_has_nan = np.isnan(array_sum)
array_has_nan

False

In [11]:
pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)

array_sum = np.sum(pos)
array_has_nan = np.isnan(array_sum)
array_has_nan


False

In [12]:
# new_pos = []
# for pos in all_pos:
#     new_pos.append(htf.new_positions(pos))
# new_pos = np.asarray(new_pos)
# new_traj = md.Trajectory(new_pos, md.Topology.from_openmm(htf._topology_proposal.new_topology))
# new_traj[0].save("debug_A501_cache_after_forward.pdb")
# new_traj.save("debug_A501_cache_after_forward.dcd")

In [13]:
context.setPeriodicBoxVectors(*box_vectors)
context.setPositions(positions)
context.setVelocitiesToTemperature(temperature)


In [43]:
positions_unit[0]

array([4.25202131, 3.04196572, 2.05178237])

In [45]:
positions_unit = np.zeros((44739, 3), )
for i, pos in enumerate(positions):
    positions_unit[i] = pos*unit.nanometer
new_pos = np.asarray(htf.new_positions(positions_unit * unit.nanometer))
new_traj = md.Trajectory(new_pos, md.Topology.from_openmm(htf._topology_proposal.new_topology))
new_traj[0].save("debug_A501_cache_after_forward_after_setting_positions.pdb")

In [14]:
# Run eq reverse (1 -> 0)
new_pos_reverse_eq = []
for _ in tqdm_notebook(range(int(nsteps_eq / 250))):
    integrator.step(250)
    pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
    new_pos_reverse_eq.append(pos)



<ipython-input-14-ba784ee5d7b8>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm_notebook(range(int(nsteps_eq / 250))):


  0%|          | 0/1000 [00:00<?, ?it/s]

OpenMMException: Particle coordinate is nan

In [ ]:
array_sum = np.sum(new_pos_reverse_eq)
array_has_nan = np.isnan(array_sum)
array_has_nan
